In [1]:
%load_ext autoreload
%autoreload 2
import netCDF4
import xarray as xr
from pathlib import Path
from hmpai.pytorch.models import *
from hmpai.training import split_data_on_participants, split_participants
from hmpai.pytorch.training import train, validate, calculate_class_weights, train_and_test, k_fold_cross_validate, test, calculate_global_class_weights, EarlyStopper
from hmpai.pytorch.utilities import DEVICE, set_global_seed, get_summary_str, save_model, load_model
from hmpai.pytorch.generators import SAT1Dataset, MultiXArrayDataset, MultiXArrayProbaDataset
from hmpai.data import SAT1_STAGES_ACCURACY, SAT_CLASSES_ACCURACY
from hmpai.visualization import plot_confusion_matrix
from hmpai.pytorch.normalization import *
from torchinfo import summary
from hmpai.utilities import print_results, CHANNELS_2D, AR_SAT1_CHANNELS
from torch.utils.data import DataLoader
from torchvision.transforms import Compose
from hmpai.pytorch.transforms import *
from hmpai.pytorch.mamba import *
from hmpai.behaviour.sat2 import read_behavioural_info
from hmpai.visualization import display_trial
from hmpai.behaviour.sat2 import SAT2_SPLITS
# from braindecode.models.eegconformer import EEGConformer
from mne.io import read_info
import os
DATA_PATH = Path(os.getenv("DATA_PATH"))

In [2]:
set_global_seed(42)

data_paths = [DATA_PATH / "sat2/stage_data_proba_250hz.nc"]

# train_percentage=100 makes test and val 100 as well
# splits = split_participants(data_paths, train_percentage=60)
splits = SAT2_SPLITS
labels = SAT_CLASSES_ACCURACY
info_to_keep = ['event_name', 'participant', 'epochs', 'rt']
whole_epoch = True
# subset_cond = 'accuracy'
subset_cond = None
add_negative = True
skip_samples = 0 # 62
cut_samples = 0 # 63

add_pe = True

In [3]:
norm_fn = norm_mad_zscore
train_data = MultiXArrayProbaDataset(
    data_paths,
    participants_to_keep=splits[0],
    normalization_fn=norm_fn,
    whole_epoch=whole_epoch,
    labels=labels,
    info_to_keep=info_to_keep,
    subset_cond=subset_cond,
    add_negative=add_negative,
    # Replace with startjittertransform
    # transform=Compose([FixedLengthCropTransform(), ReverseTimeTransform()]),
    # transform=Compose([StartJitterTransform(62, 1.0), EndJitterTransform(63, 1.0), ReverseTimeTransform(0.1), ConcatenateTransform(0.1)]),
    # transform=Compose([StartJitterTransform(62, 1.0), EndJitterTransform(63, 1.0), ReverseTimeTransform(0.1)]),
    transform=Compose([StartJitterTransform(62, 1.0), EndJitterTransform(63, 1.0)]),
    # transform=Compose([StartJitterQuadraticTransform(62, 1.0), EndJitterQuadraticTransform(63, 1.0)]),
    # transform=Compose([ReverseTimeTransform(0.5), ConcatenateTransform(0.5)]),
    # transform=Compose([ReverseTimeTransform(0.5), TimeDropoutTransform(), ConcatenateTransform(0.5)]),
    # transform=Compose([StartJitterTransform(62), FixedLengthCropTransform()]),
    skip_samples=skip_samples,
    cut_samples=cut_samples,
    add_pe=add_pe,
)
norm_vars = get_norm_vars_from_global_statistics(train_data.statistics, norm_fn)
class_weights = train_data.statistics["class_weights"]
test_data = MultiXArrayProbaDataset(
    data_paths,
    participants_to_keep=splits[1],
    normalization_fn=norm_fn,
    norm_vars=norm_vars,
    whole_epoch=whole_epoch,
    labels=labels,
    info_to_keep=info_to_keep,
    subset_cond=subset_cond,
    add_negative=add_negative,
    skip_samples=skip_samples,
    cut_samples=cut_samples,
    add_pe=add_pe,
)
val_data = MultiXArrayProbaDataset(
    data_paths,
    participants_to_keep=splits[2],
    normalization_fn=norm_fn,
    norm_vars=norm_vars,
    whole_epoch=whole_epoch,
    labels=labels,
    info_to_keep=info_to_keep,
    subset_cond=subset_cond,
    add_negative=add_negative,
    skip_samples=skip_samples,
    cut_samples=cut_samples,
    add_pe=add_pe,
)

In [6]:
# config = {'n_channels': 64,
#   'n_classes': len(labels),
#   'n_mamba_layers': 5,
#   'use_pointconv_fe': True,
#   'spatial_feature_dim': 128,
#   'use_conv': True,
#   'conv_kernel_sizes': [3, 9, 27],
#   'conv_in_channels': [128, 128, 128],
#   'conv_out_channels': [256, 256, 256],
#   'conv_concat': True,
#   'use_pos_enc': add_pe}
config = {'n_channels': 64,
  'n_classes': len(labels),
  'n_mamba_layers': 5,
  'use_pointconv_fe': True,
  'spatial_feature_dim': 128,
  'use_conv': True,
  'conv_kernel_sizes': [3, 3, 3],
  'conv_in_channels': [128, 256, 512],
  'conv_out_channels': [256, 512, 768],
  'conv_stack': True,
  'use_pos_enc': add_pe}
model = build_mamba(config)
train_and_test(
    model,
    train_data,
    test_data,
    val_data,
    logs_path=Path("../logs/"),
    workers=16,
    batch_size=32,
    labels=labels,
    lr=0.0001,
    do_spectral_decoupling=False,
    use_class_weights=False,
    class_weights=class_weights,
    whole_epoch=True,
    epochs=50,
)

  0%|          | 0/642 [00:00<?, ? batch/s]

  0%|          | 0/642 [00:00<?, ? batch/s]

  0%|          | 0/642 [00:00<?, ? batch/s]

  0%|          | 0/642 [00:00<?, ? batch/s]

  0%|          | 0/642 [00:00<?, ? batch/s]

...

  0%|          | 0/642 [00:00<?, ? batch/s]

  0%|          | 0/642 [00:00<?, ? batch/s]

  0%|          | 0/642 [00:00<?, ? batch/s]

  0%|          | 0/642 [00:00<?, ? batch/s]

[{'test_kldiv_list': [1.3765509128570557,
   1.154708743095398,
   1.4014230966567993,
   0.579967200756073,
   0.5650893449783325,
   0.6825084686279297,
   0.7388719320297241,
   0.9593201875686646,
   0.8165270090103149,
   0.5586904287338257,
   0.7333667874336243,
   0.9016988277435303,
   0.9560989141464233,
   0.6154175996780396,
   2.4202537536621094,
   1.0585191249847412,
   0.971948504447937,
   0.671795129776001,
   1.2099757194519043,
   1.1889472007751465,
   0.43571779131889343,
   3.429997682571411,
   0.7842995524406433,
   0.7963616847991943,
   0.3341410160064697,
   0.5629545450210571,
   0.4545060992240906,
   0.7919501066207886,
   0.6826044321060181,
   0.7750592827796936,
   1.5702720880508423,
   0.6025288701057434,
   0.6097281575202942,
   0.3893619179725647,
   1.3701198101043701,
   0.9925764799118042,
   1.5201363563537598,
   0.3947899043560028,
   1.1655327081680298,
   0.9923205375671387,
   0.7225768566131592,
   0.3622663617134094,
   1.89545750617980

In [ ]:
# Load in behavioural data
behaviour_sat2 = read_behavioural_info(DATA_PATH / "sat2/behavioural/df_full.csv")
test_loader_sat2 = DataLoader(
    test_data, batch_size=128, shuffle=True, num_workers=8, pin_memory=True
)
model.eval();

In [ ]:
for i in torch.randint(len(test_data), (10,)):
    print(i)
    display_trial(model, test_data, behaviour_sat2, i, labels)

In [7]:
model = base_mamba()
# 2 workers, ~18 b/s
# 4 workers, ~35 b/s
# 8 workers, ~48 b/s
# 12 workers, ~48 b/s
train_and_test(
    model,
    train_data,
    test_data,
    val_data,
    logs_path=Path("../logs/"),
    workers=8,
    batch_size=64,
    labels=labels,
    lr=0.0001,
    # lr=0.0005,
    # label_smoothing=0.1,
    # weight_decay=0.0001,
    do_spectral_decoupling=False,
    use_class_weights=False,
    class_weights=class_weights,
    whole_epoch=True,
    epochs=20,
)

  0%|          | 0/74 [00:00<?, ? batch/s]

  0%|          | 0/74 [00:00<?, ? batch/s]

  0%|          | 0/74 [00:00<?, ? batch/s]

  0%|          | 0/74 [00:00<?, ? batch/s]

  0%|          | 0/74 [00:00<?, ? batch/s]

...

  0%|          | 0/74 [00:00<?, ? batch/s]

  0%|          | 0/74 [00:00<?, ? batch/s]

  0%|          | 0/74 [00:00<?, ? batch/s]

  0%|          | 0/74 [00:00<?, ? batch/s]

KeyboardInterrupt: 

In [ ]:
from torchinfo import summary

model = MambaModel(128, 19, len(labels), 
                   5, global_pool=False, dropout=0.1)

input_shape = (1, 638, 19)
summary(model, input_shape)

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/conv.py:306: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:1031.)
  return F.conv1d(input, weight, bias, self.stride,


Layer (type:depth-idx)                   Output Shape              Param #
MambaModel                               [1, 638, 5]               4,883
├─Linear: 1-1                            [1, 638, 128]             2,560
├─Sequential: 1-2                        [1, 256, 638]             --
│    └─Conv1d: 2-1                       [1, 256, 638]             1,638,656
│    └─ReLU: 2-2                         [1, 256, 638]             --
├─Sequential: 1-3                        [1, 638, 256]             --
│    └─MambaBlock: 2-3                   [1, 638, 256]             --
│    │    └─Mamba: 3-1                   [1, 638, 256]             437,760
│    │    └─LayerNorm: 3-2               [1, 638, 256]             512
│    │    └─Dropout: 3-3                 [1, 638, 256]             --
│    └─MambaBlock: 2-4                   [1, 638, 256]             --
│    │    └─Mamba: 3-4                   [1, 638, 256]             437,760
│    │    └─LayerNorm: 3-5               [1, 638, 256]       

In [ ]:
model = SAT1GRU(64, 5)

train_and_test(
    model,
    train_data,
    test_data,
    val_data,
    logs_path=Path("../logs/"),
    workers=0,
    batch_size=128,
    labels=SAT_CLASSES_ACCURACY,
    label_smoothing=0.0001,
    weight_decay=0.01,
    lr=0.001,
    do_spectral_decoupling=False,
    use_class_weights=True,
    class_weights=class_weights,
)

In [ ]:
chk_path = Path("../models/gru100/checkpoint.pt")
checkpoint = load_model(chk_path)

model_kwargs = {
    "n_channels": len(dataset_sat1.channels),
    "n_samples": len(dataset_sat1.samples),
    "n_classes": len(dataset_sat1.labels),
}
model = SAT1GRU(**model_kwargs)
model.load_state_dict(checkpoint["model_state_dict"])
model = model.to(DEVICE)

In [17]:
test_loader = DataLoader(test_data, 128, shuffle=False, num_workers=4, pin_memory=True)
results, pred, true = test(model, test_loader, None)
pred = pred.cpu()

In [22]:
test_loader = DataLoader(
    train_dataset_sat1, 128, shuffle=True, num_workers=4, pin_memory=True
)
results, pred, true = test(model, test_loader, None)
pred = pred.cpu()

In [ ]:
plot_confusion_matrix(true, pred, SAT1_STAGES_ACCURACY[1:])